In [11]:
from langchain.document_loaders import Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from openai.embeddings_utils import get_embedding, cosine_similarity
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.redis import Redis
import os
import dotenv
import openai
import redis

# .env file must have OPENAI_API_KEY and OPENAI_API_BASE
dotenv.load_dotenv()

openai.api_type = "azure"
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = "2023-03-15-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")


In [2]:
loader = Docx2txtLoader("./data/ContosoBenefits.docx")
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 700,
    chunk_overlap  = 20,
    length_function = len,
)

texts = text_splitter.split_documents(data)
print(texts[0])

page_content='Tell me about Benefits\n\nYou can choose one of the categories below for detailed Benefits information or type your question in the chat box. \n\n\n\nPlease remember: The Annual Enrollment period for 2020 is from October 16th to October 30th, 2019.\n\nLearn What’s New for 2020\n\nI found what you need. It is in the 2020 Annual Benefits Brochure. \n\n\n\nPlease remember: The Annual Enrollment period for 2020 is from October 16th to October 30th, 2019\n\nComplete Enrollment\nOk, I can help you with that, what enrollment action do you want to take?\n\nChoose your benefits\n\nTo complete annual enrollment:\n\nOpen your benefits website. \n\nAt the pop-up window click on "Start Enrollment."' metadata={'source': './data/ContosoBenefits.docx'}


In [3]:
len(texts)

37

In [ ]:
i = 0 
for t in texts:
    # save text as txt file
    with open(f"./chunks-txt/ContosoBenefits_{texts.index(t)}.txt", "w") as f:
        f.write(t.page_content)

In [12]:
def getRedisConnection(host, access_key, port=6380, ssl=True):
    try:
        az_redis = redis.Redis(host=host, 
                            port=port, 
                            password=access_key,    
                            ssl=ssl)
        return az_redis
    except Exception as e:
        print(f'Error getRedisConnection(): {e}')
        return None
    
az_redis = getRedisConnection(host=os.getenv('REDIS_HOST'), access_key=os.getenv('REDIS_ACCESS_KEY'), port=os.getenv('REDIS_PORT'), ssl=False)


In [13]:
embeddings = OpenAIEmbeddings()

rds = Redis.from_documents(texts, embeddings,  index_name='link')

TypeError: VectorStore.from_documents() takes 3 positional arguments but 4 were given